In [ ]:
!git clone https://github.com/teachers-ai/autonomouscardata.git

Cloning into 'autonomouscardata'...
remote: Enumerating objects: 405, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 405 (delta 0), reused 6 (delta 0), pack-reused 399 (from 1)
Receiving objects: 100% (405/405), 485.63 MiB | 24.22 MiB/s, done.
Updating files: 100% (399/399), done.


In [ ]:
# prompt: write a code to read from google drive
from google.colab import drive
drive.mount('/content/drive/')
root_path = 'drive/MyDrive/raspcar/hanuman/'


Mounted at /content/drive/


In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms

class CustomImageDataset(Dataset):
    def __init__(self, root_path, transform=None):

        self.root_path = root_path
        self.transform = transform
        self.image_files = os.listdir(root_path)
        self.target = []
        # Iterate over the files and plot them
        for image_file in self.image_files:
          # Read the image file
          if  "L" in image_file :
            y = 0
          elif "F" in image_file:
            y = 1
          elif "R" in image_file:
            y = 2
          self.target.append(y)
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_path, self.image_files[idx])
        image = Image.open(img_path).resize((200, 50))
        label = self.target[idx]
        return self.transform(image), label

In [ ]:
transform = transforms.Compose([
    ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize to [-1, 1]
])

In [ ]:
dataset = CustomImageDataset(root_path, transform=transform)

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
x,y =next(iter(train_dataloader))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(16896, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
#net(x)

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001) # Use Adam optimizer

In [ ]:
# prompt: write a function for model accuracy

def model_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    accuracy = correct / len(labels)
    return accuracy


In [ ]:
for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 32:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    # calc accuracy
    with torch.no_grad():
      accuracy =[]
      for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        acc= model_accuracy(outputs, labels)
        accuracy.append(acc)

      print(f'Epoch {epoch}, Accuracy: {sum(accuracy)/len(accuracy)}')

print('Finished Training')

Epoch 0, Accuracy: 0.7617984693877551
Epoch 1, Accuracy: 0.883609693877551
Epoch 2, Accuracy: 0.9317602040816327
Epoch 3, Accuracy: 0.9451530612244898
Finished Training


In [ ]:
model_scripted = torch.jit.script(net) # Export to TorchScript
model_scripted.save('pt_ram1.pt')

In [ ]:
model_new = torch.jit.load('pt_ram1.pt')
model_new.eval()

RecursiveScriptModule(
  original_name=Net
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (pool): RecursiveScriptModule(original_name=MaxPool2d)
  (conv2): RecursiveScriptModule(original_name=Conv2d)
  (fc1): RecursiveScriptModule(original_name=Linear)
  (fc2): RecursiveScriptModule(original_name=Linear)
  (fc3): RecursiveScriptModule(original_name=Linear)
)

In [ ]:
accuracy =[]
for i, data in enumerate(train_dataloader, 0):
  inputs, labels = data
  outputs = model_new(inputs)
  acc= model_accuracy(outputs, labels)
  accuracy.append(acc)

print(f" Accuracy: {sum(accuracy)/len(accuracy)}")

 Accuracy: 0.9955357142857143


torch.Size([14, 3, 50, 200])

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize to [-1, 1]
])
#model = torch.load('hanuman1.pth', weights_only=False)

#model.eval()

image = Image.open("/content/drive/MyDrive/raspcar/hanuman/19_img_R.jpg").resize((200, 50))
image =transform(image).unsqueeze(0)
_, predicted = torch.max(model(image), 1)


In [ ]:
predicted.item()


1

In [ ]:
model = torch.jit.load('pt_model1.pt')
model.eval()
image = Image.open("/content/drive/MyDrive/raspcar/hanuman/19_img_R.jpg").resize((200, 50))
image =transform(image).unsqueeze(0)
_, predicted = torch.max(model(image), 1)

In [ ]:
#image = cv2.imread("/content/drive/MyDrive/raspcar/hanuman/19_img_R.jpg")